In [ ]:
import plom
import numpy as np

## Data

In [ ]:
np.random.seed(24)
n_x = 5
N_model = 200
N_test = 20

X = np.random.rand(n_x, N_model + N_test)
y = (X[0] * 40 + 100) + (X[1] + 5) + (X[2] * 0.01)

data = np.vstack((X, y)).T

data_model = data[:N_model]
data_test = data[-N_test:]

cond_cols = range(n_x)
qoi_col = n_x

## Bandwidth joint optimization

In [ ]:
np.random.seed()
bw_opt, mse_opt = plom.conditioning_jointly_optimal_bw(data_model, cond_cols, qoi_col, ga_workers=1, return_mse=True, logscale=True, verbose=True)
bw_silverman, mse_silverman = plom.conditioning_silverman_bw(data_model, cond_cols, qoi_col, return_mse=True)

In [ ]:
print(f"Optimal bandwidth: {bw_opt}")
print(f"Optimal MSE: {mse_opt}\n")
print(f"Silverman bandwidth: {bw_silverman}\n")
print(f"Silverman MSE: {mse_silverman}\n")

In [ ]:
x_test = data_test[:, cond_cols]
y_true = data_test[:, qoi_col]
y_pred_silverman = plom.forward_model(x_test, data_model, qoi_col, h=bw_silverman)
error_silverman = np.round(np.abs(y_pred_silverman-y_true)/np.abs(y_true)*100, 3)
y_pred_opt = plom.forward_model(x_test, data_model, qoi_col, h=bw_opt)
error_opt = np.round(np.abs(y_pred_opt-y_true)/np.abs(y_true)*100, 3)
mse_pred_opt = plom.mse(y_pred_opt, y_true)

In [ ]:
for test_case in range(N_test):
    print(f"y_true             = {y_true[test_case]}")
    print(f"y_pred (optimal)   = {y_pred_opt[test_case]} ({error_opt[test_case]}%)")
    print(f"y_pred (silverman) = {y_pred_silverman[test_case]} ({error_silverman[test_case]}%)\n")

print(f"\nPrediction MSE (optimal) = {mse_pred_opt}")

## Bandwidth marginal optimization

In [ ]:
np.random.seed()
bw_opt_marg, mse_opt_marg = plom.conditioning_marginally_optimal_bw(
    data_model, cond_cols, qoi_col, ranking_kfolds=10, opt_kfolds=1, 
    opt_cycles=10, ga_bounds=(1e-06, 1e6), shuffle=False, split_seed=None, 
    logscale=True, ga_workers=1, verbose=True)
bw_silverman, mse_silverman = plom.conditioning_silverman_bw(data_model, cond_cols, qoi_col)

In [ ]:
print(f"Optimal bandwidth: {bw_opt_marg}")
print(f"Optimal MSE: {mse_opt_marg}\n")
print(f"Silverman bandwidth: {bw_silverman}")
print(f"Silverman MSE: {mse_silverman}\n")

In [ ]:
x_test = data_test[:, cond_cols]
y_true = data_test[:, qoi_col]
y_pred_silverman = plom.forward_model(x_test, data_model, qoi_col, h=bw_silverman)
error_silverman = np.round(np.abs(y_pred_silverman-y_true)/np.abs(y_true)*100, 3)
y_pred_opt_marg = plom.forward_model(x_test, data_model, qoi_col, h=bw_opt_marg)
error_opt_marg = np.round(np.abs(y_pred_opt_marg-y_true)/np.abs(y_true)*100, 3)
mse_pred_opt_marg = plom.mse(y_pred_opt_marg, y_true)

In [ ]:
for test_case in range(N_test):
    print(f"y_true             = {y_true[test_case]}")
    print(f"y_pred (optimal)   = {y_pred_opt_marg[test_case]} ({error_opt_marg[test_case]})%")
    print(f"y_pred (silverman) = {y_pred_silverman[test_case]} ({error_silverman[test_case]})%\n")
print(f"\nPrediction MSE (optimal) = {mse_pred_opt_marg}")